In [ ]:
import os
import numpy as np
import cv2
import face_detection 
from sklearn.cluster import DBSCAN
from keras.models import load_model
from keras.applications.resnet50 import preprocess_input
import tqdm
from google.colab.patches import cv2_imshow

In [ ]:
BASE_PATH = "drive/My Drive/Social_Distancing_with_AI/"
FILE_PATH = "test_video.mp4"

In [ ]:
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

In [ ]:
mask_classifier = load_model("drive/My Drive/Social_Distancing_with_AI/Models/ResNet50_Classifier.h5")

In [ ]:
threshold_distance = 150 

In [ ]:
net = cv2.dnn.readNet(BASE_PATH+"Models/"+"yolov3.weights", BASE_PATH+"Models/"+"yolov3.cfg")


classes = []
with open(BASE_PATH+"Models/"+"coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


cap = cv2.VideoCapture(BASE_PATH + FILE_PATH )
fps = cap.get(cv2.CAP_PROP_FPS)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

os.mkdir(BASE_PATH+"Results")
os.mkdir(BASE_PATH+"Results/Extracted_Faces")
os.mkdir(BASE_PATH+"Results/Extracted_Persons")
os.mkdir(BASE_PATH+"Results/Frames")


out_stream = cv2.VideoWriter(
    BASE_PATH+'Results/Output.mp4',
    cv2.VideoWriter_fourcc('X','V','I','D'),
    fps,
    (int(width),int(height)))

print("Processing Frames :")
for frame in tqdm.notebook.tqdm(range(int(n_frames))):

    ret, img = cap.read()

    if ret == False:
        break;

    height, width, channels = img.shape

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    persons = []
    masked_faces = []
    unmasked_faces = []

   
    for i in range(len(boxes)):
        if i in indexes:

            box = np.array(boxes[i])
            box = np.where(box<0,0,box)
            (x, y, w, h) = box

            label = str(classes[class_ids[i]])

            if label=='person':

                persons.append([x,y,w,h])
                
                cv2.imwrite(BASE_PATH + "Results/Extracted_Persons/"+str(frame)
                            +"_"+str(len(persons))+".jpg",
                            img[y:y+h,x:x+w])

                
                person_rgb = img[y:y+h,x:x+w,::-1]   
                detections = detector.detect(person_rgb)

              
                if detections.shape[0] > 0:

                  detection = np.array(detections[0])
                  detection = np.where(detection<0,0,detection)

                
                  x1 = x + int(detection[0])
                  x2 = x + int(detection[2])
                  y1 = y + int(detection[1])
                  y2 = y + int(detection[3])

                  try :

                    face_rgb = img[y1:y2,x1:x2,::-1]   

                    face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)
                    face_arr = np.expand_dims(face_arr, axis=0)
                    face_arr = preprocess_input(face_arr)

                    score = mask_classifier.predict(face_arr)

                    if score[0][0]<0.5:
                      masked_faces.append([x1,y1,x2,y2])
                    else:
                      unmasked_faces.append([x1,y1,x2,y2])

                   
                    cv2.imwrite(BASE_PATH + "Results/Extracted_Faces/"+str(frame)
                                +"_"+str(len(persons))+".jpg",
                                img[y1:y2,x1:x2])

                  except:
                    continue
    
   
    person_coordinates = []

    for p in range(len(persons)):
      person_coordinates.append((persons[p][0]+int(persons[p][2]/2),persons[p][1]+int(persons[p][3]/2)))

    if len(persons)>0:
      clustering = DBSCAN(eps=threshold_distance,min_samples=2).fit(person_coordinates)
      isSafe = clustering.labels_
    else:
      isSafe = []

   
    person_count = len(persons)
    masked_face_count = len(masked_faces)
    unmasked_face_count = len(unmasked_faces)
    safe_count = np.sum((isSafe==-1)*1)
    unsafe_count = person_count - safe_count

    arg_sorted = np.argsort(isSafe)

    for i in range(1,person_count):

      if isSafe[arg_sorted[i]]!=-1 and isSafe[arg_sorted[i]]==isSafe[arg_sorted[i-1]]:
        cv2.line(img,person_coordinates[arg_sorted[i]],person_coordinates[arg_sorted[i-1]],(0,0,255),2)

    for p in range(person_count):

      a,b,c,d = persons[p]

      if isSafe[p]==-1:
        cv2.rectangle(img, (a, b), (a + c, b + d), (0,255,0), 2)
      else:
        cv2.rectangle(img, (a, b), (a + c, b + d), (0,0,255), 2)

    for f in range(masked_face_count):

      a,b,c,d = masked_faces[f]
      cv2.rectangle(img, (a, b), (c,d), (0,255,0), 2)

    for f in range(unmasked_face_count):

      a,b,c,d = unmasked_faces[f]
      cv2.rectangle(img, (a, b), (c,d), (0,0,255), 2)

 
    cv2.rectangle(img,(0,0),(width,50),(0,0,0),-1)
    cv2.rectangle(img,(1,1),(width-1,50),(255,255,255),2)

    xpos = 15

    string = "Total People = "+str(person_count)
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

    string = " ( "+str(safe_count) + " Safe "
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

    string = str(unsafe_count)+ " Unsafe ) "
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]
    
    string = "( " +str(masked_face_count)+" Masked "+str(unmasked_face_count)+" Unmasked "+\
             str(person_count-masked_face_count-unmasked_face_count)+" Unknown )"
    cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)

        out_stream.write(img)
    cv2.imwrite(BASE_PATH+"Results/Frames/"+str(frame)+".jpg",img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

out_stream.release()
cap.release()
cv2.destroyAllWindows()
print("Done !")